In [1]:
import cv2
from mtcnn import MTCNN
from PIL import Image
import numpy as np


# Process the face detection on a video
1. Load the numpy array generated from a video
1. Process the face detection
1. Visual check of the performances


## Load the numpy array : 3D + 3D numpy with colors or 3D if gray

In [2]:
USE_COLOR = True 
SHOW_DET_FACES = True
fps_np = 1.

In [3]:
# Load the numpy array 
path_arrays = "../data/arrays/"
array_name = 'TH_GNS_' + str(int(fps_np)) + 'fps'
#array_name = 'EJ_GNS_' + str(int(fps_np)) + 'fps'
if USE_COLOR:
    array_name += '_RGB'
else:
    array_name += '_Gray'
file_out =  path_arrays + array_name +'.npy'

video_np =np.load(file = file_out)

video_np = video_np.astype(np.uint8)

In [4]:
# check the size of the video 
if USE_COLOR:
    nb_frame, height, width, color = video_np.shape
else:
    nb_frame, height, width = video_np.shape
    color = 1

print('Number of frame : ' + str(nb_frame))
print('Frame size WxH : ' + str(width) + 'x' + str(height))
print('Colors : ' + str(color))


Number of frame : 153
Frame size WxH : 320x180
Colors : 3


In [5]:
def clean_screen():
    # close the window
    cv2.waitKey(1000)
    cv2.destroyAllWindows()
    cv2.waitKey(1000)
    cv2.destroyAllWindows()

## Process the video

In [18]:
# Hyperparamters 

# contour margin for the face cropping
margin = 0

# mininum size of the face to be 
min_face_size = 20 

In [19]:
detector = MTCNN(min_face_size=min_face_size)

# outputs
crop_faces = []
no_face = []
video_fd = np.copy(video_np)

for idx, frame in enumerate(video_np):

    detection_face = detector.detect_faces(frame)

    if len(detection_face) == 0:

        no_face.append(frame)

        video_fd[idx, :] = frame

    else:

        for face in detection_face:

            face_idx = face['box']
            y = int(max(face_idx[0] - margin/2, 0))
            x = int(max(face_idx[1] - margin/2, 0))
            h = face_idx[2] + margin
            w = face_idx[3] + margin

            # save the crop images
            crop_face = frame[x:x+w, y:y+h, :]
            crop_faces.append(crop_face)

            # put a block box on the faces to make the detection more visible
            # img_show[x:x+w, y:y+h, :] = 0

            video_fd[idx, x:x+w, y, :] = 255
            video_fd[idx, x:x+w, y+h, :] = 255
            video_fd[idx, x, y:y+h, :] = 255
            video_fd[idx, x+w, y:y+h, :] = 255

            video_fd[(idx,) + face['keypoints']['right_eye'][::-1]] = 255
            video_fd[(idx,) + face['keypoints']['left_eye'][::-1]] = 255
            video_fd[(idx,) + face['keypoints']['nose'][::-1]] = 255
            video_fd[(idx,) + face['keypoints']['mouth_left'][::-1]] = 255
            video_fd[(idx,) + face['keypoints']['mouth_right'][::-1]] = 255

                

In [20]:
# To generate a .gif
gif = False
if gif:
    path_arrays = "../data/arrays/"
    gif_out = path_arrays + 'TH_GNS.gif'

    imgs = [Image.fromarray(img) for img in video_fd.astype(np.uint8)]
    # duration is the number of milliseconds between frames; this is 40 frames per second
    imgs[0].save(gif_out, save_all=True, append_images=imgs[1:], duration=5, loop=0)

In [21]:
# Show the video with detected faces
idx = 0 
for frame in video_fd:
   
    cv2.imshow('face detection test', frame)
    aa = cv2.waitKey(300)
    
    if idx == 0:
        a =input('press a key to start')
    idx +=1 

press a key to start


In [22]:
clean_screen()

In [24]:
# show the frame with no face detected
print("Number of frames with no face dectected : " + str(len(no_face)) )
for face in no_face:
    cv2.namedWindow('image',cv2.WINDOW_NORMAL) 
    cv2.resizeWindow('image',width,height)
    cv2.imshow('image', face)
    aa = cv2.waitKey(100)    

Number of frames with no face dectected : 29


In [27]:
clean_screen()

In [28]:
# show the detected faces
print("Number of faces dectected : " + str(len(crop_faces)) )
for face in crop_faces:
    cv2.namedWindow('image',cv2.WINDOW_NORMAL) 
    cv2.resizeWindow('image',600,600)
    cv2.imshow('image', face)
    aa = cv2.waitKey(100)    

Number of faces dectected : 201


In [29]:
clean_screen()

In [ ]:
# TO DO : save the results
# add metadata to the  detected faces : time stamps, confidence score, keypoints
# saves the hyperparameters margin ... 